# Wine Classifier
Use the synthetic gold data set to build a classifier and guess wine.
### Bugs/Tweaks:

### Next Steps, ideas for improvement:

In [1]:
import random, sys, csv, datetime, re, pandas as pd, numpy as np, math
from pandas import DataFrame, Series, date_range, concat
import graphlab as gl;
#%matplotlib inline

[WARNING] Unable to write current GraphLab Create license to /Users/kanamyzin/.graphlab/config. Ensure that this user account has write permission to /Users/kanamyzin/.graphlab/config to save the license for offline use.
[INFO] This non-commercial license of GraphLab Create is assigned to rajivayyangar@gmail.comand will expire on January 01, 2038. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-7658 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1439946921.log
[INFO] GraphLab Server Version: 1.5.2
[WARNING] Unable to create session in specified location: '/Users/kanamyzin/.graphlab/artifacts'. Using: '/var/tmp/graphlab-kanamyzin/7658/tmp_session_3a83b8ff-f2f2-4bc5-924f-0c45a67686e4'


In [2]:
import TastingNoteParser as tnp
import GoldDataGenerator as gdg
import GoldDataFunctions as gdf

In [4]:
style_weights_dict = {'New Zealand Sauvignon Blanc':.2,
                    'California Chardonnay':.2,
                    'Alsace Pinot Gris':.2,
                     'Savennieres Chenin Blanc':.2}


In [5]:
data = gl.SFrame.read_csv('synthetic_gold_data.csv');

PROGRESS: Finished parsing file /Users/kanamyzin/Documents/wineData/SyntheticData_Guesser/synthetic_gold_data.csv
PROGRESS: Parsing completed. Parsed 80 lines in 0.042175 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,int,str,int,int,int,str,int,str,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/kanamyzin/Documents/wineData/SyntheticData_Guesser/synthetic_gold_data.csv
PROGRESS: Parsing completed. Parsed 80 lines in 0.015907 secs.


In [6]:
data.head(1)

Columns:
	X1	int
	botrytis	int
	color_concentration	str
	floral	int
	hue	str
	oak	int
	pommaceous_fruit	int
	pyrazines	int
	residual_sugar	str
	stone_fruit	int
	style_name	str
	terpenes	int
	thiols	int

Rows: 1

Data:
+----+----------+---------------------+--------+--------+-----+------------------+
| X1 | botrytis | color_concentration | floral |  hue   | oak | pommaceous_fruit |
+----+----------+---------------------+--------+--------+-----+------------------+
| 0  |    0     |        light        |   0    | yellow |  0  |        0         |
+----+----------+---------------------+--------+--------+-----+------------------+
+-----------+----------------+-------------+-----------------------------+----------+
| pyrazines | residual_sugar | stone_fruit |          style_name         | terpenes |
+-----------+----------------+-------------+-----------------------------+----------+
|     1     |      dry       |      0      | New Zealand Sauvignon Blanc |    0     |
+-----------+----------------+-------------+-----------------------------+----------+
+--------+
| thiols |
+--------+
|   1    |
+--------+
[1 rows x 13 columns]

In [9]:
data.show()

Canvas is accessible via web browser at the URL: http://localhost:64078/index.html
Opening Canvas in default web browser.


In [10]:
#train_set, test_set = data.random_split(0.9)
train_set = data
test_set = data
len(test_set)

80

In [12]:
model = gl.boosted_trees_classifier.create(
train_set, 
target = 'style_name', 
max_iterations = 1,
class_weights = style_weights_dict,
features = ['botrytis',
'color_concentration',
'floral',
'hue',
'pyrazines',
'residual_sugar',
'stone_fruit',
'terpenes',
'thiols']);

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 80
PROGRESS: Number of classes           : 4
PROGRESS: Number of feature columns   : 9
PROGRESS: Number of unpacked features : 9
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter    Accuracy Elapsed time
PROGRESS:      0   9.250e-01        0.00s


In [13]:
model.get

<bound method BoostedTreesClassifier.get of Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 80
Number of classes             : 4
Number of feature columns     : 9
Number of unpacked features   : 9

Settings
--------
Number of trees               : 4
Max tree depth                : 6
Train accuracy                : 0.925
Validation accuracy           : None
Training time (sec)           : 0.0053
>

In [14]:
predictions = model.predict(test_set)
predictions.head()

dtype: str
Rows: 10
['New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc', 'New Zealand Sauvignon Blanc']

In [15]:
model.evaluate(test_set)

{'accuracy': 0.925, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 8
 
 Data:
 +-----------------------------+-----------------------------+-------+
 |         target_label        |       predicted_label       | count |
 +-----------------------------+-----------------------------+-------+
 |    California Chardonnay    | New Zealand Sauvignon Blanc |   1   |
 |   Savennieres Chenin Blanc  |    California Chardonnay    |   1   |
 |    California Chardonnay    |    California Chardonnay    |   16  |
 |      Alsace Pinot Gris      |   Savennieres Chenin Blanc  |   1   |
 |   Savennieres Chenin Blanc  |   Savennieres Chenin Blanc  |   19  |
 |    California Chardonnay    |   Savennieres Chenin Blanc  |   3   |
 |      Alsace Pinot Gris      |      Alsace Pinot Gris      |   19  |
 | New Zealand Sauvignon Blanc | New Zealand Sauvignon Blanc |   20  |
 +-----------------------------+-----------------------------+-------+
 [8 rows x 3 columns]}

In [17]:
predictions = model.predict(test_set)
for i in range(len(predictions)):
    if (test_set[i]['style_name'] == predictions[i]):
        print predictions[i]
    else:
        print predictions[i]
        print test_set[i]

New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
New Zealand Sauvignon Blanc
Alsace Pinot Gris
Alsace Pinot Gris
Alsace Pinot Gris
Alsace Pinot Gris
Alsace Pinot Gris
Alsace Pinot Gris
Alsace Pinot Gris
Alsace Pinot Gris
Savennieres Chenin Blanc
{'hue': 'straw', 'terpenes': 0, 'pommaceous_fruit': 1, 'thiols': 0, 'color_concentration': 'moderate', 'oak': 0, 'floral': 0, 'residual_sugar': 'dry', 'style_name': 'Alsace Pinot Gris', 'stone_fruit': 1, 'botrytis': 0, 'X1': 8, 'pyrazines': 0}
Alsace Pinot Gris
Alsace Pi

In [40]:
string = 'dry, light color_concentration, straw, thiols, pyrazines'

In [41]:

def guessWine(tastingNote):
    grid=tnp.getAttributes(tastingNote)
    tasting_note = gl.SFrame(data=pd.DataFrame([grid]))
    print model.predict_topk(tasting_note)
    return grid

guessWine(string)

hue recognized as straw
color_concentration recognized as light
['dry', 'light color_concentration', 'straw', 'thiols', 'pyrazines']
+----+-----------------------------+----------------+
| id |            class            |  probability   |
+----+-----------------------------+----------------+
| 0  |    California Chardonnay    | 0.326961154212 |
| 0  | New Zealand Sauvignon Blanc | 0.253660108159 |
| 0  |   Savennieres Chenin Blanc  | 0.21089910173  |
+----+-----------------------------+----------------+
[3 rows x 3 columns]



{'botrytis': 0,
 'color_concentration': 'light',
 'floral': 0,
 'hue': 'straw',
 'oak': 0,
 'pommaceous_fruit': 0,
 'pyrazines': 0,
 'residual_sugar': 'dry',
 'stone_fruit': 0,
 'terpenes': 0,
 'thiols': 0}

In [43]:
model.show(view='Tree',tree_id=1,vlabel_hover=False)

Canvas is updated and available in a tab in the default browser.
